## Исследование надёжности заёмщиков

### Шаг 1. Изучение общей информации о данных

In [2]:
import pandas as pd
from pymystem3 import Mystem
from collections import Counter
df = pd.read_csv('/datasets/data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

Файл содержит более 20 тысяч строк.   
12 колонок, их названия менять не нужно.

### Шаг 2. Предобработка данных

### Обработка пропусков

In [3]:
# в столбце days_employed очень много отрицательных значений, заменяю их на положительные
# причина этого, как мне кажется, в том, что с файлом csv что-то случилось при выгрузке, т.к. там ещё доли дней 
df['days_employed'] = df['days_employed'].abs()
days_employed_median = df['days_employed'].median()
df['days_employed'] = df['days_employed'].fillna(days_employed_median)

total_income_median = df[df['income_type'] == 'сотрудник']['total_income'].median()
df['total_income'] = df['total_income'].fillna(total_income_median)

dob_years_mean = df['dob_years'].mean().astype('int')
df[df['dob_years'] == 0] = dob_years_mean

df['children'] = df['children'].replace(-1, 1)
df['children'] = df['children'].replace(20, 2)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

Пропущенные значения обнаружила в столбцах days_employed и total_income, трудовой стаж и доход.Тут возможно, что у человека нет стажа и дохода или он не захотел это указывать.

Пропуски заполняю медианой по всем значения в столбце дней стажа.
В столбце доходов пропуски заполню медианой по доходам сотрудников, т.к. доходы от бизнеса, пенсии или стипендии скорее всего бы указали.

Встретился также возраст равный 0, заменила его на средний возраст заёмщиков. Может быть забыли заполнить.

В столбце с количеством детей встречаются значения -1 и 20, скорее всего это опечатки. Заменила их соответственно на 1 и 2.

### Замена типа данных

In [4]:
df['days_employed'] = pd.to_numeric(df['days_employed'], errors='coerce').astype('int')
df['total_income'] = pd.to_numeric(df['total_income'], errors='coerce').astype('int')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Вывод

total_income заменяю на целое число, чтобы потом категоризировать по уровню дохода, а также заодно дни трудового стажа, хотя этот столбец не используется дальше.

Выбрала метод to_numeric, т.к. это стандартный метод и он позволяет ошибочные значения преобразовать в NaN, а astype нужен для преобразования в целое число.

### Обработка дубликатов

In [28]:
print('Количество дубликатов', df.duplicated().sum())
df['education'] = df['education'].str.lower() # переводим столбец в нижний регистр
print('Количество дубликатов после изменения регистра', df.duplicated().sum())
df.drop_duplicates(inplace=True)
df.reset_index(drop = True)
df.dropna(inplace=True) # удаляем пропуски, которые могли появится
# одна строка встретилась с неопределённым полом, заменила на женский, потому что две третьих заёмщиков женщины
df['gender'] = df['gender'].replace('XNA', 'F')

Количество дубликатов 154
Количество дубликатов после изменения регистра 171


### Вывод

Выбрала стандартный метод удаления дубликатов, а также метод для того, чтобы сбросить индексы.

Дубликаты появляются, возможно, если скопировали строку случайно несколь раз или заполнили данные одного человека, забыли и заполнили ещё раз.

Дубликатов было 154, а после приведения столбца education в нижний регист стало 171. Также к при этом появились пустые значения в строках, где было 43 - число, а не строка. Удалила их.

Дубликаты удаляем, чтобы они не меняли выводы, которые мы сделаем в итоге.

В данных есть 101 строка, заполненная числами 43. Думаю, что-то случилось с файлом при выгрузке.

### Лемматизация

In [29]:
m = Mystem()
purposes = df['purpose'].unique()

lemmas_purpose = []
for row in purposes:
    lemmas_purpose += m.lemmatize(row)

print(Counter(lemmas_purpose))   
    
def purpose(str):
    lemmas = m.lemmatize(str)
    if 'жилье' in lemmas or 'недвижимость' in lemmas:
        return 'жилье'
    if 'образование' in lemmas:
        return 'образование'
    if 'свадьба' in lemmas:
        return 'свадьба'
    if 'автомобиль' in lemmas:
        return 'автомобиль'    
df['purpose_category'] = df['purpose'].apply(purpose)

Counter({' ': 59, '\n': 38, 'покупка': 10, 'недвижимость': 10, 'автомобиль': 9, 'образование': 9, 'жилье': 7, 'с': 5, 'операция': 4, 'на': 4, 'свой': 4, 'свадьба': 3, 'строительство': 3, 'получение': 3, 'высокий': 3, 'дополнительный': 2, 'для': 2, 'коммерческий': 2, 'жилой': 2, 'заниматься': 2, 'сделка': 2, 'приобретение': 1, 'сыграть': 1, 'проведение': 1, 'семья': 1, 'собственный': 1, 'подержать': 1, 'со': 1, 'подержанный': 1, 'профильный': 1, 'сдача': 1, 'ремонт': 1})


### Вывод

В результате лемматизации мы определили четыре категории для чего человеку деньги: жилье(недвижимость), образование, свадьба, автомобиль.  
Создала список со всеми леммами, а затем посчитала их количество с помощью Counter.

### Категоризация данных

In [30]:
def family(value):
    if value in ['Не женат / не замужем', 'в разводе', 'вдовец / вдова']:
        return 'нет'
    if value in ['женат / замужем', 'гражданский брак']:
        return 'есть'


def children(value):
    if value == 0:
        return 'нет'
    if value in [1, 2]:
        return 'есть'
    if value >= 3:
        return 'много'   

def total(value):
    if value < 60000:
        return 1
    if value >= 60000 and value < 100000:
        return 2
    if value >= 100000 and value < 150000:
        return 3 
    if value >= 150000 and value < 250000:
        return 4     
    if value >= 250000:
        return 5
    
    
df['family_category'] = df['family_status'].apply(family)    
df['children_category'] = df['children'].apply(children) 

df['total_category'] = df['total_income'].apply(total) 

df = df[['children', 'debt', 'purpose_category', 'family_category', 'children_category', 'total_category']]
df.head(20)

,children,debt,purpose_category,family_category,children_category,total_category
0,1,0,жилье,есть,есть,5
1,1,0,автомобиль,есть,есть,3
2,0,0,жилье,есть,нет,3
3,3,0,образование,есть,много,5
4,0,0,свадьба,есть,нет,4
5,0,0,жилье,есть,нет,5
6,0,0,жилье,есть,нет,4
7,0,0,образование,есть,нет,3
8,2,0,свадьба,есть,есть,2
9,0,0,жилье,есть,нет,3


### Вывод

Создала столбцы с категориями: семейное положение распределила на 2 части - есть семья или нет.
children_category разделила на 3 части: нет детей, есть и многодетная семья.
В уровне дохода выделила 5 категорий по своему усмотрению.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [31]:
debt_mean = df['debt'].mean() # средняя значение по всей таблице, была ли задолженность в прошлом
print('Средняя просрочка по кредитам: {:.1%}'.format(debt_mean))
print('Наличие детей:')
print('Есть {:.1%}'.format(df[df['children'] > 0]['debt'].mean()))
print('Нет {:.1%}'.format(df[df['children'] == 0]['debt'].mean()))
print('-----------------')
print(df.groupby('children_category')['debt'].mean().map('{:.1%}'.format))

Средняя просрочка по кредитам: 8.1%
Наличие детей:
Есть 9.2%
Нет 7.5%
-----------------
children_category
есть     9.3%
много    8.2%
нет      7.5%
Name: debt, dtype: object


### Вывод

**Здесь получилось два вида.**
Люди с детьми имеют значительно больше просрочек в прошлом.
**Второй вывод:** самая большая задолженность у тех, кто имеет 1-2 ребёнка, а у многодетных среднее значение.
Думаю, это люди с высоким доходом, льготами от государства или более внимательно относятся к деньгам.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [32]:
print('Средняя просрочка по кредитам: {:.1%}'.format(debt_mean))
print('Семейное положение:')
print('Есть {:.1%}'.format(df[df['family_category'] == 'есть']['debt'].mean()))
print('Нет {:.1%}'.format(df[df['family_category'] == 'нет']['debt'].mean()))

Средняя просрочка по кредитам: 8.1%
Семейное положение:
Есть 8.0%
Нет 8.5%


### Вывод

Те, у кого есть семья возвращают кредиты с меньшим количеством задолженностей, но разница незначительная.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [33]:
print('Средняя просрочка по кредитам: {:.1%}'.format(debt_mean))
print(df.groupby('total_category')['debt'].mean().map('{:.1%}'.format))

Средняя просрочка по кредитам: 8.1%
total_category
1    6.1%
2    8.4%
3    8.5%
4    8.3%
5    6.9%
Name: debt, dtype: object


### Вывод

Люди со средним доходом имеют значительно больше задолженностей, чем остальные. Мне кажется, эти люди не слишком экономят, но и не настолько много зарабатывают, чтобы было денег с запасом.

Люди с низким доходом, скорее всего экономят и точно рассчитывают деньги.

Люди с высоки доходом много зарабатывают и имеют меньше проблем с кредитами.

- Как разные цели кредита влияют на его возврат в срок?

In [34]:
print('Средняя просрочка по кредитам: {:.1%}'.format(debt_mean))
print(df.groupby('purpose_category')['debt'].mean().map('{:.1%}'.format))

Средняя просрочка по кредитам: 8.1%
purpose_category
автомобиль     9.3%
жилье          7.2%
образование    9.3%
свадьба        8.0%
Name: debt, dtype: object


### Вывод

Меньше всего просрочек в прошлом у тех, кто покупает жильё. Думаю, люди серьёзно относятся к этому.

Больше всего просрочек у тех, кто берёт в кредит автомобиль или получает образование. Наверное, потому, что автомобиль можно разбить, а образование неизвестно окупится или нет и занимает время, вместо заработка.

Кредит на свадьбу может быть не слишком большой, здесь задолженность в прошлом чуть меньше среднего.

### Шаг 4. Общий вывод

- Наличие детей влияет на возврат в срок.

- Семейное положение влияет незначительно на возврат кредита в срок.

- Уровень дохода влияет на возврат кредита в срок если он отличается от среднего.

- Цель также влияет на возврат кредита.

Этот вывод сделала на основе данных из файла. Получается, что тут указано, были ли задолженности в прошлом. Цели кредита были другие скорее всего, могли измениться уровень дохода, семейное положение и наличие детей.